In [38]:
import tensorflow as tf
import numpy as np
import random as rd

import matplotlib.pyplot as plt
%matplotlib inline

In [46]:
image = np.loadtxt('bbs-train.txt')
label = np.loadtxt('label-train.txt')
test1 = np.loadtxt('test1/bbs-1.txt')
test2 = np.loadtxt('test2/bbs-2.txt')
test3 = np.loadtxt('test3/bbs-3.txt')
test4 = np.loadtxt('test4/bbs-4.txt')
test5 = np.loadtxt('test5/bbs-5.txt')
test6 = np.loadtxt('test6/bbs-6.txt')
test7 = np.loadtxt('test7/bbs-7.txt')
test8 = np.loadtxt('test8/bbs-8.txt')
# initialize the dataset
image_train = image[:4500]
image_val = image[4500:]
labels = np.zeros((len(label),2))
for i in range(len(label)):
    if label[i,1] == 0:
        labels[i] = [0.0, 1.0]
    if label[i,1] == 1:
        labels[i] = [1.0, 0.0]

label_train = labels[:4500]
label_val = labels[4500:]
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [101]:
x = tf.placeholder(tf.float32, shape = [None,800])
y = tf.placeholder(tf.float32, shape= [None,2])

W = tf.Variable(tf.zeros([800,2]))
b = tf.Variable(tf.zeros([2]))

y_ = tf.matmul(x, W) + b

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
## [1, 2, 2, 1]  1: input 像素 1 represents channel  1: represent output channel
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# first layer
## The convolution will compute 32 features for each 5x5 patch
## The first two dimensions are the patch size, the next is the number of input channels, 
## and the last is the number of output channels
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# reshape
## the second and third dimensions corresponding to image width and height, 
## and the final dimension corresponding to the number of color channels.
x_image = tf.reshape(x, [-1, 40, 20, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# second layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## now the dimension will reduce to 10 * 5  and 64 features, and we give 1024 nueron for 
## fully connected neuron network
W_fc1 = weight_variable([10 * 5 * 64, 1024])
b_fc1 = bias_variable([1024])

## finally flatten the input into fully connected neuron network
h_pool2_flat = tf.reshape(h_pool2, [-1, 10 * 5 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## add our softmax layer for regression above
W_fc2 = weight_variable([1024, 2])
b_fc2 = bias_variable([2])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv))

optimizer = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# *******************************************************************
# DONT Run it here or it will crash the computer. Run it on unix env.
# *******************************************************************
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        batch_x, batch_y = next_batch(50, image_train, label_train)
        if i%100 == 0:
            # for every step except for the output in training process, we wond't use dropout 
            train_accuracy = accuracy.eval(feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
            print("step %d, training accuracy %g"%(i, train_accuracy))

        # without output, we just use dropout to train the data
        optimizer.run(feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5})

    print("training complete!")
    print('test accuracy %g' % accuracy.eval(feed_dict={x: image_val, y: label_val, keep_prob: 1.0}))
    y_pred = sess.run(y_conv, feed_dict={x: image_val, keep_prob: 1.0})
    y_pred1 = sess.run(y_conv, feed_dict={x: test1, keep_prob: 1.0})
    y_pred2 = sess.run(y_conv, feed_dict={x: test2, keep_prob: 1.0})
    y_pred3 = sess.run(y_conv, feed_dict={x: test3, keep_prob: 1.0})
    y_pred4 = sess.run(y_conv, feed_dict={x: test4, keep_prob: 1.0})
    y_pred5 = sess.run(y_conv, feed_dict={x: test5, keep_prob: 1.0})
    y_pred6 = sess.run(y_conv, feed_dict={x: test6, keep_prob: 1.0})
    y_pred7 = sess.run(y_conv, feed_dict={x: test7, keep_prob: 1.0})
    y_pred8 = sess.run(y_conv, feed_dict={x: test8, keep_prob: 1.0})
    ## print(y_pred)

step 0, training accuracy 0.38
step 100, training accuracy 0.32
step 200, training accuracy 0.92
step 300, training accuracy 0.86
step 400, training accuracy 0.74
step 500, training accuracy 0.84
step 600, training accuracy 0.86
step 700, training accuracy 0.74
step 800, training accuracy 0.94
step 900, training accuracy 0.96
step 1000, training accuracy 0.92
step 1100, training accuracy 0.7
step 1200, training accuracy 0.88
step 1300, training accuracy 0.9
step 1400, training accuracy 0.76
step 1500, training accuracy 0.9
step 1600, training accuracy 0.82
step 1700, training accuracy 0.86
step 1800, training accuracy 0.82
step 1900, training accuracy 0.78
training complete!
test accuracy 1


In [104]:
## test with the test data
def ltest(data, ypred):
    for i in range(len(data)):
        if ypred[i,0] > ypred[i,1]:
            ypred[i,1] = 1.0
        else:
            ypred[i,1] = 0.0
        ypred[i,0] = i
    return ypred

res1 = ltest(test1, y_pred1)
res2 = ltest(test2, y_pred2)
res3 = ltest(test3, y_pred3)
res4 = ltest(test4, y_pred4)
res5 = ltest(test5, y_pred5)
res6 = ltest(test6, y_pred6)
res7 = ltest(test7, y_pred7)
res8 = ltest(test8, y_pred8)

list = [res1,res2,res3,res4,res5,res6,res7,res8]


for i in range(8):
    np.savetxt('test' + str(i+1) + '.csv', list[i], fmt='%d', delimiter = ',')


In [102]:
def xxx(ypred):
    for i in range(len(ypred)):
        if ypred[i,0] > ypred[i,1]:
            ypred[i,0] = 1.0
            ypred[i,1] = 0.0
        else:
            ypred[i,1] = 1.0
            ypred[i,0] = 0.0
    return ypred

ress = xxx(y_pred)


In [103]:
np.equal(ress, label_val)

array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ Tr

In [43]:
for i in range(46):
    if(y_pred[i,0] == 1):
        labeled[i,1] = 1;
    else:
        labeled[i,1] = 0;
    labeled[i,0] = i;

In [44]:
print(labeled)

[[  0.   0.]
 [  1.   0.]
 [  2.   0.]
 [  3.   0.]
 [  4.   0.]
 [  5.   0.]
 [  6.   0.]
 [  7.   0.]
 [  8.   0.]
 [  9.   0.]
 [ 10.   0.]
 [ 11.   0.]
 [ 12.   0.]
 [ 13.   0.]
 [ 14.   0.]
 [ 15.   0.]
 [ 16.   0.]
 [ 17.   0.]
 [ 18.   0.]
 [ 19.   0.]
 [ 20.   0.]
 [ 21.   0.]
 [ 22.   0.]
 [ 23.   0.]
 [ 24.   0.]
 [ 25.   0.]
 [ 26.   0.]
 [ 27.   0.]
 [ 28.   0.]
 [ 29.   0.]
 [ 30.   0.]
 [ 31.   0.]
 [ 32.   0.]
 [ 33.   0.]
 [ 34.   0.]
 [ 35.   0.]
 [ 36.   0.]
 [ 37.   0.]
 [ 38.   0.]
 [ 39.   0.]
 [ 40.   0.]
 [ 41.   0.]
 [ 42.   0.]
 [ 43.   0.]
 [ 44.   0.]
 [ 45.   0.]]
